# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")

In [5]:
import json
df_song = pd.DataFrame()
for filepath in song_files:
    with open(filepath) as f:
        data = json.load(f)
    df = pd.DataFrame([data])
    df_song = pd.concat([df, df_song])
df_song.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AREVWGE1187B9B890A,-13.442,Noci (BA),-41.9952,Bitter End,282.43546,1,SOFCHDR12AB01866EF,Living Hell,0
0,AR8IEZO1187B99055E,None,,None,Marc Shaiman,149.86404,1,SOINLJW12A8C13314C,City Slickers,2008
0,ARPFHN61187FB575F6,41.8841,"Chicago, IL",-87.6324,Lupe Fiasco,279.97995,1,SOWQTQZ12A58A7B63E,Streets On Fire (Explicit Album Version),0
0,ARLTWXK1187FB5A3F8,32.7486,"Fort Worth, TX",-97.3293,King Curtis,326.00771,1,SODREIN12A58A7F2E5,A Whiter Shade Of Pale (Live @ Fillmore West),0
0,ARULZCI1241B9C8611,None,,None,Luna Orbit Project,335.51628,1,SOSWKAV12AB018FC91,Midnight Star,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [6]:
song_data = df_song[['song_id','title','artist_id','year','duration']].values[0].tolist()
cur.execute(song_table_insert, song_data)
conn.commit()

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [7]:
artist_data = df_song[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values[0].tolist()
cur.execute(artist_table_insert, artist_data)
conn.commit()

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files("data/log_data")

In [12]:
log_data = []
for filepath in log_files:
    for line in open(filepath, 'r'):
        log_data.append(json.loads(line))
df_log = pd.DataFrame(log_data)
df_log.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [13]:
# convert the ts timestamp column to datetime
from datetime import datetime
df_log['ts'] = df_log['ts'] / 1000
df_log['dt'] = df_log['ts'].apply(datetime.fromtimestamp)

In [14]:
df_log.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,dt
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1.543537e+09,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91,2018-11-30 00:22:07.796
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1.543540e+09,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-30 01:08:41.796
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1.543540e+09,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-30 01:12:48.796
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1.543541e+09,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-30 01:17:05.796
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1.543541e+09,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-30 01:20:56.796


In [32]:
# Extract datetime data
def datetime_extract(dt):
    hr = dt.hour
    day = dt.day
    week_year = dt.strftime("%V")
    month = dt.month
    yr = dt.year
    weekday = dt.dayofweek
    list_data = [hr, day, week_year, month, yr, weekday]
    return list_data

df_log['extracted'] = df_log['dt'].apply(datetime_extract)
df_log = df_log[df_log['page']=='NextSong']
df_log.head(10)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,dt,extracted
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1.543537e+09,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91,2018-11-30 00:22:07.796,"[0, 30, 48, 11, 2018, 4]"
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1.543540e+09,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-30 01:08:41.796,"[1, 30, 48, 11, 2018, 4]"
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1.543540e+09,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-30 01:12:48.796,"[1, 30, 48, 11, 2018, 4]"
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1.543541e+09,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-30 01:17:05.796,"[1, 30, 48, 11, 2018, 4]"
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1.543541e+09,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-30 01:20:56.796,"[1, 30, 48, 11, 2018, 4]"
5,LMFAO,Logged In,Jacob,M,4,Klein,227.99628,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,I'm In Miami Bitch,200,1.543541e+09,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-30 01:24:32.796,"[1, 30, 48, 11, 2018, 4]"
6,DJ Dizzy,Logged In,Jacob,M,5,Klein,221.15220,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Sexy Bitch,200,1.543541e+09,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-30 01:28:19.796,"[1, 30, 48, 11, 2018, 4]"
7,Fish Go Deep & Tracey K,Logged In,Jacob,M,6,Klein,377.41669,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,The Cure & The Cause (Dennis Ferrer Remix),200,1.543542e+09,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-30 01:32:00.796,"[1, 30, 48, 11, 2018, 4]"
9,M83,Logged In,Jacob,M,7,Klein,96.18240,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Staring At Me,200,1.543542e+09,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-30 01:38:17.796,"[1, 30, 48, 11, 2018, 4]"
10,Saybia,Logged In,Jacob,M,8,Klein,257.25342,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,The Second You Sleep,200,1.543542e+09,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-30 01:39:53.796,"[1, 30, 48, 11, 2018, 4]"


In [27]:
# Create a dataframe
time_data = df_log['extracted'].tolist()
column_labels = ['hour', 'day', 'week_of_the_year', 'month', 'year', 'weekday']
time_df = pd.DataFrame(time_data, columns=column_labels)
time_df.head()

,hour,day,week_of_the_year,month,year,weekday
0,0,30,48,11,2018,4
1,1,30,48,11,2018,4
2,1,30,48,11,2018,4
3,1,30,48,11,2018,4
4,1,30,48,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [19]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [20]:
user_df = df_log.loc[:,["userId", "firstName", "lastName", "gender", "level"]]
user_df.head()

,userId,firstName,lastName,gender,level
0,91,Jayden,Bell,M,free
1,73,Jacob,Klein,M,paid
2,73,Jacob,Klein,M,paid
3,73,Jacob,Klein,M,paid
4,73,Jacob,Klein,M,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [21]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [22]:
songplay_df = df_log.loc[:,["userId", "level", "sessionId", "location", "userAgent"]]
songplay_df.head()

,userId,level,sessionId,location,userAgent
0,91,free,829,"Dallas-Fort Worth-Arlington, TX",Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...
1,73,paid,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
2,73,paid,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
3,73,paid,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
4,73,paid,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."


In [23]:
for index, row in df_log.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = [row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent]
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [28]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.